In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sippy import *
from sippy import functionset as fset
from sippy import functionsetSIM as fsetSIM
import control.matlab as cnt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def plot_ARX_model_pred(X_train, X_test, y_train, y_test, time_seq_train,y_pred_train, y_pred_test):
    train_len = len(X_train)
    test_len = len(X_test)

    fig = go.Figure()

    # Add shaded regions for train and test
    fig.add_shape(
        type="rect", x0=0, x1=train_len, y0=min(y_train + y_test), y1=max(y_train + y_test),
        fillcolor="#FFF2DB", opacity=0.3, layer="below", line_width=0, name="Train Region"
    )
    fig.add_shape(
        type="rect", x0=train_len, x1=train_len + test_len, y0=min(y_train + y_test), y1=max(y_train + y_test),
        fillcolor="#A9B5DF", opacity=0.3, layer="below", line_width=0, name="Test Region"
    )

    # Add vertical separator line
    fig.add_shape(
        type="line", x0=train_len, x1=train_len, y0=min(y_train + y_test), y1=max(y_train + y_test),
        line=dict(color="black", width=2, dash="dash")
    )

    # Add step plots
    fig.add_trace(go.Scatter(x=time_seq_train, y=y_train, mode='lines', line=dict(shape='hv', color='blue'), name="Train Actual"))
    fig.add_trace(go.Scatter(x=time_seq_train, y=y_pred_train, mode='lines', line=dict(shape='hv', color='red'), name="Train Prediction"))
    fig.add_trace(go.Scatter(x=list(range(train_len, train_len + test_len)), y=y_test, mode='lines', line=dict(shape='hv', color='green'), name="Test Actual"))
    fig.add_trace(go.Scatter(x=list(range(train_len, train_len + test_len)), y=y_pred_test, mode='lines', line=dict(shape='hv', color='orange'), name="Test Prediction"))
    fig.add_trace(go.Scatter(x=time_seq_train, y=X_train['L11 EK plan [MW]'], mode='lines', line=dict(shape='hv', color='black'), name="Train MW", yaxis="y2"))
    fig.add_trace(go.Scatter(x=list(range(train_len, train_len + test_len)), y= X_test['L11 EK plan [MW]'], mode='lines', line=dict(shape='hv', color='black'), name="Test MW", yaxis="y2"))

    # Layout settings
    fig.update_layout(
        title="Sys Id. L11 Heat storage energy [MWh] vs 'Heat load forecast [MW]'",
        xaxis_title="Time [15 min]",
        yaxis_title="L11 Heat storage energy Simulation [MWh]",
        yaxis2=dict(
            title="Heat Load [MW]",
            overlaying="y",
            side="right"
        ),
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),
        template="plotly_white"
    )

    # Show plot
    fig.show()


In [ ]:
# # To merge all csv files to single csv
# folder_path = r"D:\ms\January 2024\Autumn Project\Boiler\aFRR prices\LinHeat Schedules"
# print(Path(folder_path).exists())
# files = list(Path(folder_path).glob("*.csv"))  
# dfs = []
# for file in files:
#     df = pd.read_csv(file, nrows=97)  # Read rows 2 to 97
#     df.columns = df.columns.str.strip()  # Remove leading/trailing spaces in column names
#     dfs.append(df)

# merged_df = pd.concat(dfs, ignore_index=True)  # Ensures rows are stacked correctly

# # Save the merged DataFrame
# merged_df.to_csv("merged_output.csv", index=False)

In [ ]:
folder_path = r"D:\ms\January 2024\Thesis\Boiler-Bidding-model\Datasets\LinHeat Schedule January.csv"
january_data = pd.read_csv(folder_path)

In [3]:
# drop the timestamps and only keep the rows with Mwh data
dataset = january_data.dropna(subset=['L11 Heat storage energy [MWh]'])
dataset = dataset.reset_index(drop=True)
# create train and test data set
dataset = dataset[['Heat load forecast [MW]','L11 Heat storage energy [MWh]']]
dataset["Heat load forecast [MW]"] = dataset["Heat load forecast [MW]"].shift(-1)
dataset = dataset.iloc[:-2]
X_train, X_test, y_train, y_test = train_test_split(dataset['Heat load forecast [MW]'], dataset['L11 Heat storage energy [MWh]'], test_size=0.5, shuffle=False)
# do we need a separate validation set? 

In [ ]:
# 2nd order ARX Model
na_ords = [2]
nb_ords = [[2]]
theta = [[0]]
id_ARX = system_identification(y_train, X_train, 'ARX', ARX_orders= [na_ords, nb_ords, theta])
G = id_ARX.G
num = id_ARX.NUMERATOR
den = id_ARX.DENOMINATOR

time_seq_train = np.arange(len(X_train)) #time sequence for the predictor
time_seq_test = np.arange(len(X_test))
time_seq = np.arange(len(dataset))


k = 1 # steps ahead for prediction
y_pred_train = fset.validation(id_ARX, X_train, y_train, Time=np.arange(len(X_train)), k=k)
y_pred_test = fset.validation(id_ARX, X_test, y_test, Time=np.arange(len(X_test)), k=k)
y_pred_train = np.squeeze(y_pred_train)
y_pred_test = np.squeeze(y_pred_test)


from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = mse ** 0.5

print(f"MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}")


In [28]:
"""A = id_ARMAX.DENOMINATOR  # A(z) coefficients (denominator)
B = id_ARMAX.NUMERATOR    # B(z) coefficients (numerator for inputs)
C = id_ARMAX.C      # C(z) coefficients (error term)"""
A = np.array(den)
B = np.array(num)
A = A.reshape(1, 3) 
B = B.reshape(1, 2)
print('Transfer fn\n', G)
print('B: \n', B)
print('A: \n', A)


Transfer fn
 
  0.03319 z + 0.09114
-----------------------
z^2 - 1.052 z + 0.07153

dt = 1.0

B: 
 [[0.03318858 0.09114499]]
A: 
 [[ 1.         -1.05169693  0.07153242]]


In [ ]:
C = np.zeros((A.shape[0], A.shape[0])) 
D = np.zeros((A.shape[0], B.shape[0])) 
"""x,y=fsetSIM.SS_lsim_process_form(Identified_system.A, Identified_system.B, Identified_system.C,Identified_system.D, u,Identified_system.x0)"""
x, y_pred = fsetSIM.SS_lsim_process_form(A, B, C, D, x_values)

In [ ]:
# Apply the ARX model to schedule for a day
folder_path = r"D:\ms\January 2024\Thesis\Boiler-Bidding-model\Datasets\boiler-12025022508_00_17_LHP_2_PHLIT_export.csv"
testfile = pd.read_csv(folder_path)
testfile["Heat load forecast [MW]"] = testfile["Heat load forecast [MW]"].shift(-1)
testfile = testfile.iloc[:-1]
x_values = testfile['Heat load forecast [MW]']
x_values = x_values.values
time = np.arange(0, len(x_values), 1)
y_pred, Time, Xsim = cnt.lsim(id_ARX.G, x_values, time)

In [52]:
testfile['predicted MWh'] = y_pred


fig = make_subplots(rows=1, cols=1,shared_xaxes=True,vertical_spacing=0.1,subplot_titles=('Time Series Plot'),specs=[[{'secondary_y': True}]]  )

fig.add_trace(go.Scatter(x=testfile['UTC'], y=testfile['Heat load forecast [MW]'], mode='lines', name='Heat load forecast [MW]'),secondary_y=False)

fig.add_trace(go.Scatter(x=testfile['UTC'], y=testfile['L11 EK plan [MW]'], mode='lines', name='L11 EK plan [MW]'),secondary_y=False)

fig.add_trace(go.Scatter(x=testfile['UTC'], y=testfile['L11 Heat storage energy [MWh]'], mode='lines', name='L11 Heat storage energy [MWh]'),secondary_y=True)

fig.add_trace(go.Scatter(x=testfile['UTC'], y=testfile['predicted MWh'], mode='lines', name='predicted MWh'),secondary_y=True)

fig.update_layout(
    title_text="Time Series of Heat Load Forecast, EK Plan, and Heat Storage Energy",
    xaxis_title="UTC",
    yaxis_title="MW (Heat load forecast & EK plan)",
    yaxis2_title="MWh (Heat storage energy)",
    template="plotly",
)

fig.show()


In [ ]:
# Model order selection
n_range = range(1,7)
results = []
for na in n_range:
    na_ords = [na]
    nb_ords = [[na]]
    theta = [[0]]
    id_ARX = system_identification(y_train, X_train, 'ARX', ARX_orders= [na_ords, nb_ords, theta])
    G = id_ARX.G
    num = id_ARX.NUMERATOR
    den = id_ARX.DENOMINATOR
    k = 1 # steps ahead for prediction
    y_pred_train = fset.validation(id_ARX, X_train, y_train, Time=np.arange(len(X_train)), k=k)
    y_pred_test = fset.validation(id_ARX, X_test, y_test, Time=np.arange(len(X_test)), k=k)
    y_pred_train = np.squeeze(y_pred_train)
    y_pred_test = np.squeeze(y_pred_test)
    mae = mean_absolute_error(y_test, y_pred_test)
    mse = mean_squared_error(y_test, y_pred_test)
    rmse = mse ** 0.5
    results.append({"model": (na, na, 0), "G": G, "num": num, "den": den, "MAE": mae, "MSE": mse, "RMSE": rmse})


error = pd.DataFrame(results)
print(error[['model', 'MAE', 'MSE']])

plt.plot(error['model'].str[0], error['MAE'])

Train ARMAX model with two inputs Heat load forecast [MW] and L11 EK plan [MW]

In [ ]:
# drop the timestamps and only keep the rows with Mwh data
dataset2 = january_data.dropna(subset=['L11 Heat storage energy [MWh]'])
dataset2 = dataset2.reset_index(drop=True)
# create train and test data set
dataset2 = dataset2[['Heat load forecast [MW]','L11 Heat storage energy [MWh]','L11 EK plan [MW]']]
dataset2["Heat load forecast [MW]"] = dataset2["Heat load forecast [MW]"].shift(-1)
dataset2["L11 EK plan [MW]"] = dataset2["L11 EK plan [MW]"].shift(-1)
dataset2 = dataset2.iloc[:-1]

X_train, X_test, y_train, y_test = train_test_split(dataset2[['Heat load forecast [MW]','L11 EK plan [MW]']], dataset2['L11 Heat storage energy [MWh]'], test_size=0.5, shuffle=False)
# do we need a separate validation set? 

In [ ]:
# 2nd order ARMAX Model with two inputs
# Define the orders for the ARMAX model
na_ords = [2] # Order of A(z) (output)
nb_ords = [[2, 2]] # Orders of B(z) for each input (2 inputs, so 2 orders)
nc_ords = [2]  # Order of C(z) (error model)
theta = [[0, 0]]  # Time delays for each input (2 inputs, so 2 delays)
id_ARMAX = system_identification(y_train, X_train, 'ARMAX', ARMAX_orders= [na_ords, nb_ords, nc_ords, theta])
G = id_ARMAX.G
num = id_ARMAX.NUMERATOR
den = id_ARMAX.DENOMINATOR


time_seq_train = np.arange(len(X_train)) #time sequence for the predictor
time_seq_test = np.arange(len(X_test))
time_seq = np.arange(len(dataset2))


k = 1 # steps ahead for prediction
y_pred_train = fset.validation(id_ARMAX, X_train, y_train, Time=np.arange(len(X_train)), k=k)
y_pred_test = fset.validation(id_ARMAX, X_test, np.zeros(len(X_test)), Time=np.arange(len(X_test)), k=k)
y_pred_train = np.squeeze(y_pred_train)
y_pred_test = np.squeeze(y_pred_test)


from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = mse ** 0.5

print(f"MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}")
%tb

MAE: 39.7397, MSE: 1929.3343, RMSE: 43.9242


No traceback available to show.


In [10]:
plot_ARX_model_pred(X_train, X_test, y_train, y_test, time_seq_train)